In [4]:
import altair as alt
import pandas as pd
pd.set_option('display.max_columns', None)

In [8]:
migration_df = pd.read_csv('migration_flow.csv')
migration_df.sample(5)

,STRUCTURE,STRUCTURE_ID,STRUCTURE_NAME,ACTION,REF_AREA,Reference area,CITIZENSHIP,Citizenship,FREQ,Frequency of observation,MEASURE,Measure,SEX,Sex,BIRTH_PLACE,Place of birth,EDUCATION_LEV,Education level,UNIT_MEASURE,Unit of measure,TIME_PERIOD,Time period,OBS_VALUE,Observation value,OBS_STATUS,Observation status,UNIT_MULT,Unit multiplier,DECIMALS,Decimals
176012,DATAFLOW,OECD.ELS.IMD:DSD_MIG@DF_MIG(1.0),International migration database,I,CZE,Czechia,MKD,North Macedonia,A,Annual,B12,Outflows of foreign population,_T,Total,_Z,Not applicable,_Z,Not applicable,PS,Persons,2019.0,NaN,142.0,NaN,A,Normal value,0.0,Units,0,Zero
77797,DATAFLOW,OECD.ELS.IMD:DSD_MIG@DF_MIG(1.0),International migration database,I,SWE,Sweden,STP,Sao Tome and Principe,A,Annual,B12,Outflows of foreign population,_T,Total,_Z,Not applicable,_Z,Not applicable,PS,Persons,2007.0,NaN,0.0,NaN,A,Normal value,0.0,Units,0,Zero
23025,DATAFLOW,OECD.ELS.IMD:DSD_MIG@DF_MIG(1.0),International migration database,I,NZL,New Zealand,EST,Estonia,A,Annual,B12,Outflows of foreign population,_T,Total,_Z,Not applicable,_Z,Not applicable,PS,Persons,2004.0,NaN,2.0,NaN,A,Normal value,0.0,Units,0,Zero
135586,DATAFLOW,OECD.ELS.IMD:DSD_MIG@DF_MIG(1.0),International migration database,I,SVK,Slovak Republic,MAR,Morocco,A,Annual,B11,Inflows of foreign population,_T,Total,_Z,Not applicable,_Z,Not applicable,PS,Persons,2005.0,NaN,3.0,NaN,A,Normal value,0.0,Units,0,Zero
209862,DATAFLOW,OECD.ELS.IMD:DSD_MIG@DF_MIG(1.0),International migration database,I,FIN,Finland,BLZ,Belize,A,Annual,B12,Outflows of foreign population,_T,Total,_Z,Not applicable,_Z,Not applicable,PS,Persons,2020.0,NaN,0.0,NaN,A,Normal value,0.0,Units,0,Zero


# Data Cleanup

In [ ]:

columns_to_drop = [
    "STRUCTURE", "STRUCTURE_ID", "STRUCTURE_NAME", "ACTION",
    "FREQ", "Frequency of observation", "SEX", "Sex",
    "BIRTH_PLACE", "Place of birth", "EDUCATION_LEV", "Education level",
    "OBS_STATUS", "Observation status", "UNIT_MULT", "Unit multiplier",
    "DECIMALS", "Decimals", "UNIT_MEASURE", "Unit of measure", "Time period", "Observation value"
]

df_cleaned = migration_df.drop(columns=columns_to_drop)


In [54]:
df_cleaned.sample(5)

,REF_AREA,Reference area,CITIZENSHIP,Citizenship,MEASURE,Measure,TIME_PERIOD,OBS_VALUE
184205,ESP,Spain,KEN,Kenya,B11,Inflows of foreign population,2005.0,56.0
47262,EST,Estonia,IRN,Iran,B11,Inflows of foreign population,2017.0,33.0
203103,KOR,Korea,SLV,El Salvador,B12,Outflows of foreign population,2005.0,10.0
52937,HUN,Hungary,IRL,Ireland,B11,Inflows of foreign population,1998.0,12.0
19173,USA,United States,FRA,France,B11,Inflows of foreign population,2015.0,5215.0


# Creating distinct datasets for inbound and outbound migration

In [184]:

inflow_df = df_cleaned[df_cleaned['Measure'] == 'Inflows of foreign population']
inflow_df.sample(5)

,REF_AREA,Reference area,CITIZENSHIP,Citizenship,MEASURE,Measure,TIME_PERIOD,OBS_VALUE
13466,SVN,Slovenia,HTI,Haiti,B11,Inflows of foreign population,2019.0,0.0
78615,AUS,Australia,PRI,Puerto Rico,B11,Inflows of foreign population,2016.0,0.0
219220,CAN,Canada,ROU,Romania,B11,Inflows of foreign population,2022.0,480.0
56943,CAN,Canada,KOR,Korea,B11,Inflows of foreign population,1997.0,4001.0
9210,AUS,Australia,SVN,Slovenia,B11,Inflows of foreign population,2004.0,31.0


In [186]:
inflow_df = inflow_df.fillna(0)
inflow_df["TIME_PERIOD"] = inflow_df["TIME_PERIOD"].astype(int)

In [187]:
inflow_df = inflow_df[inflow_df['TIME_PERIOD'] != 0]

In [188]:
inflows_pivot = inflow_df.pivot_table(
    index="Reference area",
    columns="TIME_PERIOD",
    values="OBS_VALUE",
    aggfunc="sum")

In [191]:
inflows_pivot.columns = inflows_pivot.columns.astype(str)
inflows_pivot.fillna(0, inplace=True)

In [192]:
inflows_pivot.sample(5)

TIME_PERIOD,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
Reference area,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Finland,13034.0,13556.0,17625.0,18180.0,17335.0,19896.0,23805.0,21629.0,20670.0,24885.0,27617.0,29892.0,37493.0,42324.0,38089.0,36115.0,40565.0,46668.0,47746.0,47294.0,42828.0,54548.001,47470.0,46292.0,48289.067,46495.0,56085.0,84939.0
New Zealand,108126.0,82109.0,62843.0,52240.0,59018.0,74906.0,171310.0,204911.0,173383.0,144670.0,147722.0,161499.0,168928.0,180487.0,151962.0,139802.0,143581.0,143496.0,150822.0,181366.0,203032.0,208046.000,206853.0,208445.0,249484.000,108993.0,56355.0,177696.0
Norway,39532.0,42114.0,54838.0,66724.0,75363.0,65317.0,60304.0,70888.0,61624.0,63863.0,71550.0,85441.0,120753.0,134018.0,126763.0,129294.0,141326.0,139857.0,133737.0,122714.0,117979.0,116828.000,99382.0,88644.0,88989.000,61547.0,93108.0,166259.0
Denmark,65653.0,43661.0,40765.0,42145.0,40154.0,45104.0,48828.0,42703.0,36689.0,37281.0,40202.0,47893.0,62816.0,73937.0,63844.0,66804.0,69079.0,99142.0,111835.0,128994.0,150679.0,142406.000,132786.0,129089.0,123813.000,95976.0,114485.0,202291.0
Slovak Republic,6706.0,5432.0,6102.0,6445.0,5861.0,4622.0,4723.0,4784.0,10089.0,17854.0,17837.0,11432.0,17189.0,17613.0,13087.0,8594.0,7611.0,5880.0,4950.0,4836.0,7548.0,7220.001,5822.0,5738.0,4972.000,5668.0,4004.0,3564.0


In [200]:
outflow_df = df_cleaned[df_cleaned['Measure'] == 'Outflows of foreign population']
outflow_df.sample(5)

,REF_AREA,Reference area,CITIZENSHIP,Citizenship,MEASURE,Measure,TIME_PERIOD,OBS_VALUE
186265,SVN,Slovenia,ALB,Albania,B12,Outflows of foreign population,2009.0,10.0
153604,GBR,United Kingdom,SYR,Syrian Arab Republic,B12,Outflows of foreign population,1998.0,0.0
40845,FIN,Finland,AGO,Angola,B12,Outflows of foreign population,2009.0,1.0
134241,LTU,Lithuania,EST,Estonia,B12,Outflows of foreign population,2018.0,2.0
110850,ESP,Spain,RWA,Rwanda,B12,Outflows of foreign population,2012.0,3.0


In [201]:
outflow_df = outflow_df.fillna(0)
outflow_df["TIME_PERIOD"] = outflow_df["TIME_PERIOD"].astype(int)

In [202]:
outflows_pivot = outflow_df.pivot_table(
    index="Reference area",
    columns="TIME_PERIOD",
    values="OBS_VALUE",
    aggfunc="sum"
)

In [203]:
outflows_pivot.reset_index(inplace=True)
outflows_pivot.fillna(0, inplace=True)

In [204]:
outflows_pivot.sample(5)

TIME_PERIOD,Reference area,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
22,Poland,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,50405.0,48599.0,51040.0,68910.0,49515.0,68653.0,89480.0,40070.0,45897.0,43067.0,43976.0,57484.0,72903.0,0.0
24,Slovak Republic,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8009.0,11049.0,2265.0,1217.0,1289.0,912.0,2928.0,1682.0,909.0,160.0,76.0,138.0,70.0,254.0,62.0,64.0,78.0,18.0,4.0,14.0
18,Luxembourg,8832.0,11120.0,11541.0,13420.0,13776.0,19575.0,21418.0,22991.0,19157.0,20704.0,20166.0,21641.0,17282.0,15909.0,14596.0,15195.0,14875.0,17105.0,17771.0,18812.0,20747.0,22558.0,23043.0,23248.0,25983.0,24062.0,25771.0,0.0
25,Slovenia,0.0,0.0,0.0,11319.0,3136.0,3840.0,6455.0,9071.0,7599.0,11443.0,12737.0,21520.0,22728.0,14561.0,28527.0,21640.0,13739.0,11692.0,10791.0,12005.0,12002.0,12699.0,14400.0,13055.0,16198.0,28564.0,15323.0,0.0
20,New Zealand,13162.0,15417.0,17697.0,19316.0,19057.0,31248.0,54624.0,54502.0,68121.0,76733.0,76139.0,73266.0,74257.0,79184.0,82414.0,87594.0,90324.0,82947.0,74620.0,71412.0,73987.0,78339.0,90618.0,94130.0,110111.0,78606.0,88065.0,87423.0


In [206]:
from vega_datasets import data

world = data.world_110m.url

In [207]:
countries_df = pd.read_csv("countries.csv")
countries_df.head()

,country,latitude,longitude,name
0,AD,42.546245,1.601554,Andorra
1,AE,23.424076,53.847818,United Arab Emirates
2,AF,33.939110,67.709953,Afghanistan
3,AG,17.060816,-61.796428,Antigua and Barbuda
4,AI,18.220554,-63.068615,Anguilla


In [208]:
iso_url = 'https://raw.githubusercontent.com/lukes/ISO-3166-Countries-with-Regional-Codes/master/all/all.csv'
iso_df = pd.read_csv(iso_url)

iso_df.head()

,name,alpha-2,alpha-3,country-code,iso_3166-2,region,sub-region,intermediate-region,region-code,sub-region-code,intermediate-region-code
0,Afghanistan,AF,AFG,4,ISO 3166-2:AF,Asia,Southern Asia,NaN,142.0,34.0,NaN
1,Åland Islands,AX,ALA,248,ISO 3166-2:AX,Europe,Northern Europe,NaN,150.0,154.0,NaN
2,Albania,AL,ALB,8,ISO 3166-2:AL,Europe,Southern Europe,NaN,150.0,39.0,NaN
3,Algeria,DZ,DZA,12,ISO 3166-2:DZ,Africa,Northern Africa,NaN,2.0,15.0,NaN
4,American Samoa,AS,ASM,16,ISO 3166-2:AS,Oceania,Polynesia,NaN,9.0,61.0,NaN


In [209]:
world = alt.topo_feature(data.world_110m.url, 'countries')

base_map = (
    alt.Chart(world)
    .mark_geoshape(fill="white", stroke="gray")
    .properties(width=600, height=300)
    .project("naturalEarth1")
)
base_map

alt.Chart(...)